In [151]:
import os
import os.path as osp
from argparse import ArgumentParser
from typing import Dict, List, Tuple, Union
import copy
import pandas as pd
import torch
# from atom3d.datasets import deserialize
from Bio.PDB import PDBParser
import re
import pickle
import io
import json
import gzip

from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from tqdm import tqdm
import logging
import lmdb
from pathlib import Path
import numpy as np
import openbabel
from openbabel import pybel

from utils import MyData, prot_graph_transform, hetero_graph_transform

pybel.ob.obErrorLog.SetOutputLevel(0)
atomic_num_dict = lambda x: {1: 'H', 2: 'HE', 3: 'LI', 4: 'BE', 5: 'B', 6: 'C', 7: 'N', 8: 'O', 9: 'F', 11: 'NA',
                   15: 'P', 16: 'S', 17: 'Cl', 20:'Ca', 25: 'MN', 26: 'FE', 30: 'ZN', 35: 'Br', 53: 'I', 80: 'Hg'}.get(x, 'Others')



    
class GNNTransformAffinity(object):
    def __init__(
        self,
        cutoff: float = 4.5,
        remove_hydrogens: bool = True,
        max_num_neighbors: int = 32,
        supernode: bool = False,
        offset_strategy: int = 0,
        task='affinity', #lba/ppi
        gearnet=False
    ):
        self.cutoff = cutoff
        self.remove_hydrogens = remove_hydrogens
        self.max_num_neighbors = max_num_neighbors
        self.supernode = supernode
        self.offset_strategy = offset_strategy
        self.task = task
        self.gearnet = gearnet

    def __call__(self, item: Dict) -> MyData:
        # print("Using Transform Affinity")
        ligand_df = item["atoms_ligand"]
        protein_df = item["atoms_protein"]

        if isinstance(ligand_df, pd.DataFrame):
            atom_df = pd.concat([protein_df, ligand_df], axis=0)
            if self.remove_hydrogens:
                # remove hydrogens
                atom_df = atom_df[atom_df.element != "H"].reset_index(drop=True)
            lig_flag = torch.zeros(atom_df.shape[0], dtype=torch.long)
            lig_flag[-len(ligand_df):] = 0
        else:
            atom_df = protein_df
            if self.remove_hydrogens:
                # remove hydrogens
                atom_df = atom_df[atom_df.element != "H"].reset_index(drop=True)
            lig_flag = torch.zeros(atom_df.shape[0], dtype=torch.long)
        chain_ids = list(set(protein_df['chain']))
        for i, id in enumerate(chain_ids):
            lig_flag[torch.tensor(list(atom_df['chain'] == id))] = i + 1
        labels = item["labels"]
        #目前是按照肽链来区分不同的蛋白，为了便于Unprot分类
        if self.gearnet:
            graph = hetero_graph_transform(
                atom_df=atom_df, super_node=self.supernode, flag=lig_flag, protein_seq=item['protein_seq']
            )
        else:
            graph = prot_graph_transform(
                atom_df=atom_df, cutoff=self.cutoff, max_num_neighbors=self.max_num_neighbors, flag=lig_flag, super_node=self.supernode, offset_strategy=self.offset_strategy
            )
        
        lba = labels['lba']
        ppi = labels['ppi']
        if lba != -1:
            affinity = lba
            graph.affinity_mask = torch.ones(1)
            graph.y = torch.FloatTensor([affinity])
        elif ppi != -1:
            affinity = ppi
            graph.affinity_mask = torch.ones(1)
            graph.y = torch.FloatTensor([affinity])
        else:
            graph.y = torch.FloatTensor([0])
            graph.affinity_mask = torch.zeros(1)
        graph.chains = lig_flag[lig_flag!=0]

        graph.lig_flag = lig_flag
        graph.prot_id = item["complex_id"]
        graph.type = self.task
        return graph
    
    
class GNNTransformEC(object):
    def __init__(
        self,
        cutoff: float = 4.5,
        remove_hydrogens: bool = True,
        max_num_neighbors: int = 32,
        supernode: bool = False,
        offset_strategy: int = 0,
        task='ec', #ec
        gearnet=False
    ):
        self.cutoff = cutoff
        self.remove_hydrogens = remove_hydrogens
        self.max_num_neighbors = max_num_neighbors
        self.supernode = supernode
        self.offset_strategy = offset_strategy
        self.task = task
        self.gearnet = gearnet

    def __call__(self, item: Dict) -> MyData:
        # print("Using Transform EC")
        info_root = './output_info/uniprot_dict_all.json'
        with open(info_root, 'r') as f:
            chain_uniprot_info = json.load(f)
        ligand_df = item["atoms_ligand"]
        protein_df = item["atoms_protein"]
        atom_df = protein_df
        if isinstance(ligand_df, pd.DataFrame):
            atom_df = pd.concat([protein_df, ligand_df], axis=0)
            if self.remove_hydrogens:
                # remove hydrogens
                atom_df = atom_df[atom_df.element != "H"].reset_index(drop=True)
            lig_flag = torch.zeros(atom_df.shape[0], dtype=torch.long)
            lig_flag[-len(ligand_df):] = 0
        else:
            atom_df = protein_df
            if self.remove_hydrogens:
                # remove hydrogens
                atom_df = atom_df[atom_df.element != "H"].reset_index(drop=True)
            lig_flag = torch.zeros(atom_df.shape[0], dtype=torch.long)
        chain_ids = list(set(protein_df['chain']))
        uniprot_ids = []
        labels = item["labels"]
        pf_ids = []
        #目前是按照肽链来区分不同的蛋白，为了便于Unprot分类
        for i, id in enumerate(chain_ids):
            lig_flag[torch.tensor(list(atom_df['chain'] == id))] = i + 1
            if '-' in item['complex_id']:
                pf_ids.append(0)
                break
            if id in chain_uniprot_info[item['complex_id']]:
                uniprot_id = chain_uniprot_info[item['complex_id']][id]
                uniprot_ids.append(uniprot_id)
                labels_uniprot = labels['uniprots']
                if uniprot_id in labels_uniprot:
                    for idx, u in enumerate(labels_uniprot):
                        if uniprot_id == u:
                            pf_ids.append(idx)
                            break
                else:
                    pf_ids.append(-1)
                    print("Error, you shouldn't come here!")
            else:
                pf_ids.append(-1)
        num_classes =538
        if self.gearnet:
            graph = hetero_graph_transform(
                atom_df=atom_df, super_node=self.supernode, flag=lig_flag, protein_seq=item['protein_seq']
            )
        else:
            graph = prot_graph_transform(
                atom_df=atom_df, cutoff=self.cutoff, max_num_neighbors=self.max_num_neighbors, flag=lig_flag, super_node=self.supernode, offset_strategy=self.offset_strategy
            )
        ec = labels['ec']
        graph.functions = []
        graph.valid_masks = []
        for i, pf_id in enumerate(pf_ids):
            if pf_id == -1:
                valid_mask = torch.zeros(num_classes)
                prop = torch.zeros(num_classes)
                graph.functions.append(prop)
                graph.valid_masks.append(valid_mask)
                continue
            valid_mask = torch.ones(num_classes)
            annotations = []
            ec_annot = ec[pf_id]
            if ec_annot == -1:
                valid_mask[:] = 0
            else:
                annotations = ec_annot
                
            prop = torch.zeros(num_classes).scatter_(0,torch.tensor(annotations),1)
            graph.functions.append(prop)
            graph.valid_masks.append(valid_mask)
        try:
            graph.functions = torch.vstack(graph.functions)
            graph.valid_masks = torch.vstack(graph.valid_masks)
        except:
            print("PF ids:", pf_ids)
            print(item['complex_id'], chain_ids, labels)
            print(len(graph.functions))
            print(pf_ids)
            print(graph.functions)
            raise RuntimeError    
        graph.chains = lig_flag[lig_flag!=0]
        # print(item['complex_id'])

        graph.lig_flag = lig_flag
        if len(chain_ids) != len(graph.functions):
            print(item['complex_id'])
            print(chain_ids)
            print(len(chain_ids), len(graph.functions))
        graph.prot_id = item["complex_id"]
        graph.type = self.task
        # print("Task Type:", graph.type)
        return graph



class GNNTransformMultiTask(object):
    def __init__(
        self,
        cutoff: float = 4.5,
        remove_hydrogens: bool = True,
        max_num_neighbors: int = 32,
        supernode: bool = False,
        offset_strategy: int = 0,
        gearnet = False
    ):
        self.cutoff = cutoff
        self.remove_hydrogens = remove_hydrogens
        self.max_num_neighbors = max_num_neighbors
        self.supernode = supernode
        self.offset_strategy = offset_strategy
        self.gearnet = gearnet

    def __call__(self, item: Dict) -> MyData:
        print("Using Transform LBA")
        info_root = './output_info/uniprot_dict_all.json'
        with open(info_root, 'r') as f:
            chain_uniprot_info = json.load(f)
        
        ligand_df = item["atoms_ligand"]
        protein_df = item["atoms_protein"]

        if isinstance(ligand_df, pd.DataFrame):
            atom_df = pd.concat([protein_df, ligand_df], axis=0)
            print('atom_df shape: ',atom_df.shape)
            if self.remove_hydrogens:
                # remove hydrogens
                atom_df = atom_df[atom_df.element != "H"].reset_index(drop=True)
            print('protein_df shape: ',protein_df.shape)
            print('ligand_df shape: ',ligand_df.shape)
            print('atom_df shape: ',atom_df.shape)
            lig_flag = torch.zeros(atom_df.shape[0], dtype=torch.long)
            lig_flag[-len(ligand_df):] = 0
            print('lig_flag shape: ',lig_flag.shape)
            print(atom_df)
        else:
            atom_df = protein_df
            print('protein_df shape: ',protein_df.shape)
            # print('ligand_df shape: ',atom_df.shape)
            print('atom_df shape: ',atom_df.shape)
            if self.remove_hydrogens:
                # remove hydrogens
                atom_df = atom_df[atom_df.element != "H"].reset_index(drop=True)
            lig_flag = torch.zeros(atom_df.shape[0], dtype=torch.long)
            print('lig_flag shape: ',lig_flag.shape)
        chain_ids = list(set(protein_df['chain']))
        uniprot_ids = []
        labels = item["labels"]
        pf_ids = []
        #目前是按照肽链来区分不同的蛋白，为了便于Unprot分类
        for i, id in enumerate(chain_ids):
            lig_flag[torch.tensor(list(atom_df['chain'] == id))] = i + 1
            if '-' in item['complex_id']:
                pf_ids.append(0)
                break
            if id in chain_uniprot_info[item['complex_id']]:
                uniprot_id = chain_uniprot_info[item['complex_id']][id]
                uniprot_ids.append(uniprot_id)
                labels_uniprot = labels['uniprots']
                if uniprot_id in labels_uniprot:
                    for idx, u in enumerate(labels_uniprot):
                        if uniprot_id == u:
                            pf_ids.append(idx)
                            break
                else:
                    pf_ids.append(-1)
                    print("Error, you shouldn't come here!")
            else:
                pf_ids.append(-1)
        # ec, mf, bp, cc
        # num_classes = 538 + 490 + 1944 + 321
        num_classes = [538, 490, 1944, 321]
        total_classes = 538 + 490 + 1944 + 321
        # 找个办法把chain和Uniprot对应起来，然后就可以查了
        if self.gearnet:
            graph = hetero_graph_transform(
                atom_df=atom_df, super_node=self.supernode, flag=lig_flag, protein_seq=item['protein_seq']
            )
        else:
            graph = prot_graph_transform(
                atom_df=atom_df, cutoff=self.cutoff, max_num_neighbors=self.max_num_neighbors, flag=lig_flag, super_node=self.supernode, offset_strategy=self.offset_strategy
            )
        lba = labels['lba']
        ppi = labels['ppi']
        ec = labels['ec']
        go = labels['go']
        graph.affinities = torch.FloatTensor([lba, ppi]).unsqueeze(0)
        if lba != -1:
            graph.affinity_mask = torch.tensor([1, 0]).unsqueeze(0)
        elif ppi != -1:
            graph.affinity_mask = torch.tensor([0, 1]).unsqueeze(0)
        else:
            graph.affinity_mask = torch.tensor([0, 0]).unsqueeze(0)

        graph.functions = []
        graph.valid_masks = []
        for i, pf_id in enumerate(pf_ids):
            if pf_id == -1:
                valid_mask = torch.zeros(len(num_classes))
                prop = torch.zeros(total_classes)
                graph.functions.append(prop)
                graph.valid_masks.append(valid_mask)
                continue
            valid_mask = torch.ones(len(num_classes))
            annotations = []
            ec_annot = ec[pf_id]
            go_annot = go[pf_id]
            if ec_annot == -1:
                valid_mask[0] = 0
            else:
                annotations = annotations + ec_annot
            if go_annot == -1:
                valid_mask[1:] = 0
            else:
                mf_annot = go_annot['molecular_functions'] 
                mf_annot = [j + 538 for j in mf_annot]
                if len(mf_annot) == 0:
                    valid_mask[1] = 0
                bp_annot = go_annot['biological_process']
                bp_annot = [j + 538 + 490 for j in bp_annot]
                if len(bp_annot) == 0:
                    valid_mask[2] = 0
                cc_annot = go_annot['cellular_component']
                cc_annot = [j + 538 + 490 + 1944 for j in cc_annot]
                if len(cc_annot) == 0:
                    valid_mask[3] = 0
                annotations = annotations + mf_annot + bp_annot + cc_annot
                
            prop = torch.zeros(total_classes).scatter_(0,torch.tensor(annotations),1)
            graph.functions.append(prop)
            graph.valid_masks.append(valid_mask)
        try:
            graph.functions = torch.vstack(graph.functions)
            graph.valid_masks = torch.vstack(graph.valid_masks)
        except:
            print("PF ids:", pf_ids)
            print(item['complex_id'], chain_ids, labels)
            print(len(graph.functions))
            print(pf_ids)
            print(graph.functions)
            raise RuntimeError
        graph.chains = lig_flag[lig_flag!=0]
        # print(item['complex_id'])

        graph.lig_flag = lig_flag
        print(graph.chains.shape)
        print(graph.lig_flag.shape)
        print(graph.chains.tolist())
        print(graph.lig_flag.tolist())
        if len(chain_ids) != len(graph.functions):
            print(item['complex_id'])
            print(chain_ids)
            print(len(chain_ids), len(graph.functions))
        graph.prot_id = item["complex_id"]
        graph.type = 'multi'
        return graph

In [152]:
class CustomMultiTaskDataset(Dataset):
    """
    The Custom MultiTask Dataset with uniform labels
    """
    def __init__(self, root_dir: str = './datasets/Multi_task', label_dir: str = './datasets/MultiTask/uniformed_labels.json',
                remove_hoh = True, remove_hydrogen = False, cutoff = 6, split : str = 'train', task = 'multi', gearnet = False, alpha_only=False):
        super(CustomMultiTaskDataset, self).__init__(root_dir)
        print("Initializing MultiTask Dataset...")
        self.root_dir = root_dir
        self.cache_dir = os.path.join(root_dir, "{}.cache".format(split))
        with open(label_dir, 'r') as f:
            self.labels = json.load(f)
        self.remove_hoh = remove_hoh
        self.remove_hydrogen = remove_hydrogen # 移除氢
        self.cutoff = cutoff
        self.gearnet = gearnet
        self.alpha_only = alpha_only
        file_dir = os.path.join(root_dir, split+'.txt')        
        self.ec_root = './data/EnzymeCommission/all'
        self.go_root = './data/GeneOntology/all'
        self.lba_root = './data/PDBbind/refined-set'
        self.pp_root = './data/PDBbind/PP'
        with open(file_dir, 'r') as f:
            self.files = f.readlines()
            self.files = [i.strip() for i in self.files]
        if split not in ['train', 'val', 'test', 'train_all','train_ec', 'val_ec', 'test_ec']:
            print("Wrong selected split. Have to choose between ['train', 'val', 'test', 'test_all']")
            print("Exiting code")
            exit()
        if task not in ['affinity', 'ec', 'cc', 'mf', 'bp', 'multi', 'go', 'ppi', 'lba']:
            print("Wrong selected task. Have to choose between ['affinity', 'ec', 'cc', 'mf', 'bp', 'multi', 'go']")
            print("Exiting code")
            exit()
        self.split = split
        self.task = task
        self.process_complexes()
        print('finish init')
    def find_structure(self, item):
        self.ec_files = os.listdir(self.ec_root)
        self.go_files = os.listdir(self.go_root)
        self.lba_files = os.listdir(self.lba_root)
        self.pp_files = os.listdir(self.pp_root)
        if '-' in item:
            if item+'.pdb' in self.ec_files:
                return os.path.join(self.ec_root, item+'.pdb'), -1
            elif item+'.pdb' in self.go_files:
                return os.path.join(self.go_root, item+'.pdb'), -1
        else:
            if item + '.ent.pdb' in self.pp_files:
                return os.path.join(self.pp_root, item+'.ent.pdb'), -1
            elif item in self.lba_files:
                protein_dir = os.path.join(self.lba_root, item, item + "_protein.pdb")
                ligand_dir = os.path.join(self.lba_root, item, item + '_ligand.mol2')
                return protein_dir, ligand_dir
        print(item)
        return -1, -1
    def gen_df(self, coords, elements):
        assert len(coords) == len(elements)
        unified_elements = []
        xs, ys, zs = [coord[0] for coord in coords], [coord[1] for coord in coords], [coord[2] for coord in coords]
        for item in elements:
            if item in ['CL', 'Cl', 'Br', 'BR', 'AT', 'At', 'F', 'I']:
                element = 'Halogen'
            elif item in ['FE', 'ZN', 'MG', 'MN', 'K', 'LI', 'Ca', 'Hg']:
                element = 'Metal'
            else:
                element = item
            unified_elements.append(element)
        df = pd.DataFrame({'element': unified_elements, 'x': xs, 'y': ys, 'z': zs})
        return df
    def process_complexes(self):
        p = PDBParser(QUIET=True)
        nmr_files = []
        wrong_number = []
        self.processed_complexes = []
        corrupted = []
        # cache_dir = os.path.join(self.root_dir, self.cache_dir)
        if os.path.exists(self.cache_dir):
            print("Start loading cached Multitask files...")
            self.processed_complexes = pickle.load(open(self.cache_dir, 'rb'))
            print("Complexes Before Checking:", self.len())
            self.check_dataset()
            print("Complexes Before Task Selection:", self.len())
            self.choose_task_items()
            print("Dataset size:", self.len())
            if self.alpha_only:
                print("Only retaining Alpha Carbon atoms for the atom_df")
                self.retain_alpha_carbon()
        else:
            print("Cache not found! Start processing Multitask files...Total Number {}".format(len(self.files)))
            # count = 0
            for score_idx, item in enumerate(tqdm(self.files)):
                structure_dir, ligand_dir = self.find_structure(item)
                if ligand_dir != -1:
                    ligand = next(pybel.readfile('mol2', ligand_dir))
                    ligand_coords = [atom.coords for atom in ligand]
                    atom_map_lig = [atomic_num_dict(atom.atomicnum) for atom in ligand]
                    ligand_df = self.gen_df(ligand_coords, atom_map_lig)
                else:
                    ligand_df = None
                try:
                    structure = p.get_structure(item, structure_dir)
                except:
                    corrupted.append(item)
                    continue
                # structure = p.get_structure(item, structure_dir)
                compound_info = structure.header['compound']
                protein_numbers = len(compound_info.items())
                
                if len(structure) > 1:
                    nmr_files.append(item)
                    continue
                if item not in self.labels:
                    wrong_number.append(item)
                    continue
                model = structure[0]
                chains = list(model.get_chains())
                pattern = re.compile(r'\d+H.')
                processed_complex = {'complex_id': item, 'num_proteins': protein_numbers, 'labels': self.labels[item],
                                    'atoms_protein': [], 'protein_seq': [], 'atoms_ligand':ligand_df}
                elements = []
                xs = []
                ys = []
                zs = []
                chain_ids = []
                protein_seq = []
                names = []
                resnames = []
                # chain = chains[0]
                for chain in chains:
                    if chain.id == ' ':
                        continue
                    for residue in chain.get_residues():
                        # 删除HOH原子
                        if self.remove_hoh and residue.get_resname() == 'HOH':
                            continue
                        protein_seq.append(residue.get_resname())
                        for atom in residue:
                            # 删除氢原子
                            atom_id = atom.get_id()
                            if self.remove_hydrogen and atom.get_id().startswith('H') or pattern.match(atom.get_id()) != None:
                                continue
                            if atom_id.startswith('H') or pattern.match(atom.get_id()) != None:
                                element = 'H'
                            elif atom_id[0:2] in ['CL', 'Cl', 'Br', 'BR', 'AT', 'At']:
                                element = 'Halogen'
                            elif atom_id[0:2] in ['FE', 'ZN', 'MG', 'MN', 'K', 'LI']:
                                element = 'Metal'
                            elif atom_id[0] in ['F', 'I']:
                                element = 'Halogen'
                            elif atom_id[0] in ['C','N','O','S','P']:
                                element = atom_id[0]
                            else:
                                element = atom_id
                            names.append(atom_id)
                            elements.append(element)
                            chain_ids.append(chain.id)
                            resnames.append(residue.get_resname())
                            x, y, z = atom.get_vector()
                            xs.append(x)
                            ys.append(y)
                            zs.append(z)
                protein_df = pd.DataFrame({'chain': chain_ids, 'resname': resnames, 'element': elements, 'name': names, 'x': xs, 'y': ys, 'z': zs})
                processed_complex['atoms_protein'] = protein_df
                processed_complex['protein_seq'] = protein_seq
                
                self.processed_complexes.append(processed_complex)
                # count += 1
                # if count == 128:
                #     break
            print("Structure processed Done, dumping...")
            print("Structures with Wrong numbers:", len(wrong_number), wrong_number)
            print("Structures with NMR methods:", len(nmr_files), nmr_files)
            print("Corrupted:", len(corrupted), corrupted)
            pickle.dump(self.processed_complexes, open(self.cache_dir, 'wb'))
            print("Complexes Before Checking:", self.len())
            self.check_dataset()
            print("Complexes Before Task Selection:", self.len())
            self.choose_task_items()
            print("Dataset size:", self.len())
            if self.alpha_only:
                print("Only retaining Alpha Carbon atoms for the atom_df")
                self.retain_alpha_carbon()
    def correctness_check(self, chain_uniprot_info, complx):
        #Annotation Correctness Check
        correct = True
        chain_ids = list(set(complx['atoms_protein']['chain']))
        if '-' not in complx['complex_id']:
            for i, id in enumerate(chain_ids):
                if id in chain_uniprot_info[complx['complex_id']]:
                    uniprot_id = chain_uniprot_info[complx['complex_id']][id]
                    labels_uniprot = complx['labels']['uniprots']
                    if uniprot_id not in labels_uniprot:
                        print("Error, you shouldn't come here!")
                        correct = False
                        print(complx['complex_id'], chain_ids, chain_uniprot_info[complx['complex_id']])
        return correct
    def cal_length_thres(self, complxes):
        length_list = []
        for complx in complxes:
            length = len(complx['atoms_protein']['element'])
            length_list.append(length)
        sorted_list = sorted(length_list)
        thres = sorted_list[int(0.95*len(sorted_list))]
        print("Cutting Threshold of atom numbers:", thres)
        return thres
    def length_check(self, complx, thres):
        if len(complx['atoms_protein']['element']) > thres:
            return False
        else:
            return True
    def check_dataset(self):
        print("Checking the dataset...")
        info_root = './output_info/uniprot_dict_all.json'
        with open(info_root, 'r') as f:
            chain_uniprot_info = json.load(f)
        thres = self.cal_length_thres(self.processed_complexes)
        if self.split == 'train':
            thres = 6712
        self.processed_complexes = [i for i in tqdm(self.processed_complexes) if self.length_check(i, thres) and self.correctness_check(chain_uniprot_info, i)]
    
    def retain_alpha_carbon(self):
        new_complexes = []
        for item in self.processed_complexes:
            protein_df = item['atoms_protein']
            # print("Original Nodes:", len(protein_df))
            new_protein_df = protein_df[protein_df.name == 'CA'].reset_index(drop=True)
            item['atoms_protein'] = new_protein_df
            # print("Retaining Alpha Carbons:", len(new_protein_df))
            new_complexes.append(item)
        self.processed_complexes = new_complexes
    
    def choose_task_items(self):
        # 根据不同的任务，训练单独的模型
        if self.split in ['train', 'val', 'test']:
            extra_dir = './datasets/MultiTask/{}.txt'.format(self.split)
            with open(extra_dir, 'r') as f:
                extra_info = f.readlines()
                extra_info = [i.strip() for i in extra_info]
        else:
            extra_info = []
        if self.task == 'ec':
            print("Using Enzyme Commission Dataset for training:")
            root_dir = './output_info/enzyme_commission_uniprots.json'
            with open(root_dir, 'r') as f:
                info_dict = json.load(f)
            new_complexes = []
            for item in self.processed_complexes:
                if item['complex_id'] in info_dict or item['complex_id'] in extra_info: #.keys()?
                    labels = item['labels']
                    annot_number = len(labels['uniprots'])
                    for j in range(annot_number):
                        labels['go'][j] = -1
                    item['labels'] = labels
                    new_complexes.append(item)
            self.processed_complexes = new_complexes
            self.transform_func = GNNTransformEC(task=self.task, gearnet=self.gearnet)
            print("Using EC dataset and transformation")
        elif self.task in ['bp', 'mf', 'cc', 'go']:
            print("Using Gene Ontology {} Dataset for training:".format(self.split))
            root_dir = './output_info/gene_ontology_uniprots.json'
            with open(root_dir, 'r') as f:
                info_dict = json.load(f)
            new_complexes = []
            for item in self.processed_complexes:
                if item['complex_id'] in info_dict or item['complex_id'] in extra_info:
                    labels = item['labels']
                    annot_number = len(labels['uniprots'])
                    for j in range(annot_number):
                        labels['ec'][j] = -1
                    item['labels'] = labels
                    new_complexes.append(item)
            self.processed_complexes = new_complexes
            self.transform_func = GNNTransformGO(task=self.task, gearnet=self.gearnet)
        elif self.task in ['affinity', 'lba', 'ppi']:
            print("Using Affinity Dataset for training:")
            root_dir = './output_info/protein_protein_uniprots.json'
            with open(root_dir, 'r') as f:
                info_dict1 = json.load(f)
            root_dir2 = './output_info/protein_ligand_uniprots.json'
            with open(root_dir2, 'r') as f:
                info_dict2 = json.load(f)
            info_dict = {**info_dict1, **info_dict2}
            new_complexes = []
            for item in self.processed_complexes:
                if item['complex_id'] in info_dict or item['complex_id'] in extra_info:
                    labels = item['labels']
                    annot_number = len(labels['uniprots'])
                    for j in range(annot_number):
                        labels['ec'][j] = -1
                        labels['go'][j] = -1
                    item['labels'] = labels
                    new_complexes.append(item)
            self.processed_complexes = new_complexes
            self.transform_func = GNNTransformAffinity(task=self.task, gearnet=self.gearnet)
        else:
            self.transform_func = GNNTransformMultiTask(gearnet=self.gearnet)
    def len(self):
        return len(self.processed_complexes)
    def get(self, idx):
        return self.transform_func(self.processed_complexes[idx])
    def print_complexes(self, idx):
        return self.processed_complexes[idx]

In [153]:
my_functions = None
class GNNTransformGO(object):
    def __init__(
        self,
        cutoff: float = 4.5,
        remove_hydrogens: bool = True,
        max_num_neighbors: int = 32,
        supernode: bool = False,
        offset_strategy: int = 0,
        task='bp', #可能是bp, mf, cc中的一个
        gearnet=False
    ):
        self.cutoff = cutoff
        self.remove_hydrogens = remove_hydrogens
        self.max_num_neighbors = max_num_neighbors
        self.supernode = supernode
        self.offset_strategy = offset_strategy
        self.task = task
        self.gearnet = gearnet
        # print("GNNtransforgo")

    def __call__(self, item: Dict) -> MyData:
        info_root = './output_info/uniprot_dict_all.json'
        with open(info_root, 'r') as f:
            chain_uniprot_info = json.load(f)
        # print("Using Transform {}".format(self.task))
        ligand_df = item['atoms_ligand']
        protein_df = item["atoms_protein"]
        atom_df = protein_df
        if isinstance(ligand_df, pd.DataFrame):
            atom_df = pd.concat([protein_df, ligand_df], axis=0)
            if self.remove_hydrogens:
                # remove hydrogens
                atom_df = atom_df[atom_df.element != "H"].reset_index(drop=True)
            lig_flag = torch.zeros(atom_df.shape[0], dtype=torch.long)
            lig_flag[-len(ligand_df):] = 0
        else:
            atom_df = protein_df
            if self.remove_hydrogens:
                # remove hydrogens
                atom_df = atom_df[atom_df.element != "H"].reset_index(drop=True)
            lig_flag = torch.zeros(atom_df.shape[0], dtype=torch.long)
        chain_ids = list(set(protein_df['chain']))
        uniprot_ids = []
        labels = item["labels"]
        pf_ids = []
        #目前是按照肽链来区分不同的蛋白，为了便于Unprot分类
        for i, id in enumerate(chain_ids):
            lig_flag[torch.tensor(list(atom_df['chain'] == id))] = i + 1
            if '-' in item['complex_id']:
                pf_ids.append(0)
                break
            if id in chain_uniprot_info[item['complex_id']]:
                uniprot_id = chain_uniprot_info[item['complex_id']][id]
                uniprot_ids.append(uniprot_id)
                labels_uniprot = labels['uniprots']
                if uniprot_id in labels_uniprot:
                    for idx, u in enumerate(labels_uniprot):
                        if uniprot_id == u:
                            pf_ids.append(idx)
                            break
                else:
                    pf_ids.append(-1)
                    print("Error, you shouldn't come here!")
            else:
                pf_ids.append(-1)
        # print("pf_ids", pf_ids)        
        if self.task == 'mf':
            num_classes = 490
        elif self.task == 'bp':
            num_classes = 1944
        elif self.task == 'cc':
            num_classes = 321
        elif self.task == 'go':
            num_classes = 490 + 1944 + 321
        else:
            raise RuntimeError
        # 找个办法把chain和Uniprot对应起来，然后就可以查了
        if self.gearnet:
            graph = hetero_graph_transform(
                atom_df=atom_df, super_node=self.supernode, flag=lig_flag, protein_seq=item['protein_seq']
            )
        else:
            graph = prot_graph_transform(
                atom_df=atom_df, cutoff=self.cutoff, max_num_neighbors=self.max_num_neighbors, flag=lig_flag, super_node=self.supernode, offset_strategy=self.offset_strategy
            )
        go = labels['go']
        # graph.y = torch.zeros(self.num_classes).scatter_(0,torch.tensor(labels),1)
        graph.functions = []
        graph.valid_masks = []
        
        for i, pf_id in enumerate(pf_ids):
            if pf_id == -1:
                valid_mask = torch.zeros(num_classes)
                prop = torch.zeros(num_classes)
                graph.functions.append(prop)
                graph.valid_masks.append(valid_mask)
                continue
            valid_mask = torch.ones(num_classes)
            annotations = []
            go_annot = go[pf_id]
            if self.task == 'mf':
                mf_annot = go_annot['molecular_functions'] 
                mf_annot = [j for j in mf_annot]
                if len(mf_annot) == 0:
                    valid_mask[:] = 0
                annotations = mf_annot
            elif self.task == 'bp':
                bp_annot = go_annot['biological_process']
                bp_annot = [j for j in bp_annot]
                if len(bp_annot) == 0:
                    valid_mask[:] = 0
                annotations = bp_annot
            elif self.task == 'cc':
                cc_annot = go_annot['cellular_component']
                cc_annot = [j for j in cc_annot]
                if len(cc_annot) == 0:
                    valid_mask[:] = 0
                annotations = cc_annot
            elif self.task == 'go':
                mf_annot = go_annot['molecular_functions'] 
                mf_annot = [j for j in mf_annot]
                if len(mf_annot) == 0:
                    valid_mask[: 490] = 0
                bp_annot = go_annot['biological_process']
                bp_annot = [j + 490 for j in bp_annot]
                if len(bp_annot) == 0:
                    valid_mask[490: 490+1944] = 0
                cc_annot = go_annot['cellular_component']
                cc_annot = [j+490+1944 for j in cc_annot]
                if len(cc_annot) == 0:
                    valid_mask[490+1944: ] = 0
                annotations = mf_annot + bp_annot + cc_annot
                
            prop = torch.zeros(num_classes).scatter_(0,torch.tensor(annotations),1)
            graph.functions.append(prop)
            graph.valid_masks.append(valid_mask)
        try:
            # print(annotations)
            
            # my_functions = copy.deepcopy(graph.functions)
            # print(my_functions)
            graph.functions = torch.vstack(graph.functions)
            graph.valid_masks = torch.vstack(graph.valid_masks)
            # print("386",graph.functions[0,386])
        except:
            print("PF ids:", pf_ids)
            print(item['complex_id'], chain_ids, labels)
            print(len(graph.functions))
            print(pf_ids)
            print(graph.functions)
            raise RuntimeError
    
        graph.chains = lig_flag[lig_flag!=0]

        graph.lig_flag = lig_flag
        if len(chain_ids) != len(graph.functions):
            print(item['complex_id'])
            print(chain_ids)
            print(len(chain_ids), len(graph.functions))
        print("functions len",len(graph.functions))
        print("functions", graph.functions)
        graph.prot_id = item["complex_id"]
        graph.type = self.task
        print(graph.lig_flag.tolist())
        print(graph.chains.tolist())
        return graph

In [154]:
import os
import os.path as osp
from argparse import ArgumentParser
from typing import Dict, List, Tuple, Union
import gzip
import pandas as pd
import torch
# from atom3d.datasets import deserialize
from Bio.PDB import PDBParser
import re
import pickle
import io
import json
import gzip

from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from tqdm import tqdm
import logging
import lmdb
from pathlib import Path
import numpy as np
import openbabel
from openbabel import pybel

from utils import MyData, prot_graph_transform, hetero_graph_transform

pybel.ob.obErrorLog.SetOutputLevel(0)
atomic_num_dict = lambda x: {1: 'H', 2: 'HE', 3: 'LI', 4: 'BE', 5: 'B', 6: 'C', 7: 'N', 8: 'O', 9: 'F', 11: 'NA',
                   15: 'P', 16: 'S', 17: 'Cl', 20:'Ca', 25: 'MN', 26: 'FE', 30: 'ZN', 35: 'Br', 53: 'I', 80: 'Hg'}.get(x, 'Others')

class CustomMultiTaskDataset(Dataset):
    """
    The Custom MultiTask Dataset with uniform labels
    """
    def __init__(self, root_dir: str = './datasets/MultiTask', label_dir: str = './datasets/MultiTask/uniformed_labels.json',
                remove_hoh = True, remove_hydrogen = False, cutoff = 6, split : str = 'train', task = 'multi', gearnet = False, alpha_only=False):
        super(CustomMultiTaskDataset, self).__init__(root_dir)
        print("in")
        print("in Initializing MultiTask Dataset...")
        self.root_dir = root_dir
        self.cache_dir = os.path.join(root_dir, "{}.cache".format(split))
        with open(label_dir, 'r') as f:
            self.labels = json.load(f)
        self.remove_hoh = remove_hoh
        self.remove_hydrogen = remove_hydrogen # 移除氢
        self.cutoff = cutoff
        self.gearnet = gearnet
        self.alpha_only = alpha_only
        file_dir = os.path.join(root_dir, split+'.txt')        
        self.ec_root = './data/EnzymeCommission/all'
        # self.ec_root = './data/EC/all'
        self.go_root = './data/GeneOntology/all'
        # self.go_root = './data/GO/all'
        self.lba_root = './data/PDBbind/refined-set'
        self.pp_root = './data/PDBbind/PP'
        with open(file_dir, 'r') as f:
            self.files = f.readlines()
            self.files = [i.strip() for i in self.files]
        if split not in ['train', 'val', 'test', 'train_all','train_ec', 'val_ec', 'test_ec']:
            print("Wrong selected split. Have to choose between ['train', 'val', 'test', 'test_all']")
            print("Exiting code")
            exit()
        if task not in ['affinity', 'ec', 'cc', 'mf', 'bp', 'multi', 'go', 'lba', 'ppi']:
            print("Wrong selected task. Have to choose between ['affinity', 'ec', 'cc', 'mf', 'bp', 'multi', 'go']")
            print("Exiting code")
            exit()
        self.split = split
        self.task = task
        self.process_complexes()
    def find_structure(self, item):
        self.ec_files = os.listdir(self.ec_root)
        self.go_files = os.listdir(self.go_root)
        self.lba_files = os.listdir(self.lba_root)
        self.pp_files = os.listdir(self.pp_root)
        pdb_files = os.listdir('./data/EC/all')
        if '-' in item:
            if item +'.pdb' in self.ec_files:
                return os.path.join(self.ec_root, item+'.pdb'), -1
            elif item+'.pdb' in self.go_files:
                return os.path.join(self.go_root, item+'.pdb'), -1
            elif item.split('-')[0] +'.pdb.gz' in self.ec_files:
                return os.path.join(self.ec_root, item.split('-')[0] +'.pdb.gz'), -1
            elif item.split('-')[0] +'.pdb' in self.ec_files:
                return os.path.join(self.ec_root, item.split('-')[0] +'.pdb'), -1
            elif item.split('-')[0] +'.pdb.gz' in self.go_files:
                return os.path.join(self.go_root, item.split('-')[0] +'.pdb.gz'), -1
            elif item.split('-')[0] +'.pdb' in self.go_files:
                return os.path.join(self.go_root, item.split('-')[0] +'.pdb'), -1
            elif item.split('-')[0] +'.pdb.gz' in pdb_files:
                return os.path.join('./data/EC/all', item.split('-')[0] +'.pdb.gz'), -1
            elif item.split('-')[0] +'.pdb' in pdb_files:
                return os.path.join('./data/EC/all', item.split('-')[0] +'.pdb'), -1
        else:
            if item + '.ent.pdb' in self.pp_files:
                return os.path.join(self.pp_root, item+'.ent.pdb'), -1
            elif item in self.lba_files:
                protein_dir = os.path.join(self.lba_root, item, item + "_protein.pdb")
                ligand_dir = os.path.join(self.lba_root, item, item + '_ligand.mol2')
                return protein_dir, ligand_dir
        print("item", item)
        return -1, -1
    def gen_df(self, coords, elements):
        assert len(coords) == len(elements)
        unified_elements = []
        xs, ys, zs = [coord[0] for coord in coords], [coord[1] for coord in coords], [coord[2] for coord in coords]
        for item in elements:
            if item in ['CL', 'Cl', 'Br', 'BR', 'AT', 'At', 'F', 'I']:
                element = 'Halogen'
            elif item in ['FE', 'ZN', 'MG', 'MN', 'K', 'LI', 'Ca', 'Hg']:
                element = 'Metal'
            else:
                element = item
            unified_elements.append(element)
        df = pd.DataFrame({'element': unified_elements, 'x': xs, 'y': ys, 'z': zs})
        return df
    def process_complexes(self):
        p = PDBParser(QUIET=True)
        nmr_files = []
        wrong_number = []
        self.processed_complexes = []
        corrupted = []
        # cache_dir = os.path.join(self.root_dir, self.cache_dir)
        if os.path.exists(self.cache_dir):
            print("Start loading cached Multitask files...")
            self.processed_complexes = pickle.load(open(self.cache_dir, 'rb'))
            print("Complexes Before Checking:", self.len())
            self.check_dataset()
            print("Complexes Before Task Selection:", self.len())
            self.choose_task_items()
            print("Dataset size:", self.len())
            if self.alpha_only:
                print("Only retaining Alpha Carbon atoms for the atom_df")
                self.retain_alpha_carbon()
        else:
            print("Cache not found! Start processing Multitask files...Total Number {}".format(len(self.files)))
            count = 0
            for score_idx, item in enumerate(tqdm(self.files)):
                structure_dir, ligand_dir = self.find_structure(item)
                if ligand_dir != -1:
                    ligand = next(pybel.readfile('mol2', ligand_dir))
                    ligand_coords = [atom.coords for atom in ligand]
                    atom_map_lig = [atomic_num_dict(atom.atomicnum) for atom in ligand]
                    ligand_df = self.gen_df(ligand_coords, atom_map_lig)
                else:
                    ligand_df = None
                split_chain_flag = False
                # print(structure_dir)
                file_is_chain = False
                have_chain_id = False
                pdb_id = None
                chain_id = None
                try:
                    if '-' in structure_dir:
                        file_is_chain = True
                    if '.gz' in structure_dir:
                        if '-' in item:
                            pdb_id, chain_id = item.split('-')
                            file_is_chain = False
                            have_chain_id = True
                        file_handle = gzip.open(structure_dir, 'rt')
                        split_chain_flag = True
                    else:
                        file_handle = structure_dir
                    structure = p.get_structure(item, file_handle)
                except:
                    corrupted.append(item)
                    continue
                # structure = p.get_structure(item, structure_dir)
                compound_info = structure.header['compound']
                protein_numbers = len(compound_info.items())
                
                if len(structure) > 1:
                    nmr_files.append(item)
                    continue
                if item not in self.labels:
                    wrong_number.append(item)
                    continue
                model = structure[0]
                chains = list(model.get_chains())
                if file_is_chain:
                    chain = chains[0]
                else:
                    if have_chain_id:
                        for chain in chains:
                            if chain.id == chain_id:
                                break
                if have_chain_id:
                    if chain.id != chain_id:
                        print('cannot find chain:', pdb_id, chain_id)
                chains = [chain]
                pattern = re.compile(r'\d+H.')
                processed_complex = {'complex_id': item, 'num_proteins': protein_numbers, 'labels': self.labels[item],
                                    'atoms_protein': [], 'protein_seq': [], 'atoms_ligand':ligand_df}
                elements = []
                xs = []
                ys = []
                zs = []
                chain_ids = []
                protein_seq = []
                names = []
                resnames = []
                # chain = chains[0]
                for chain in chains:
                    if chain.id == ' ':
                        continue
                    for residue in chain.get_residues():
                        # 删除HOH原子
                        if self.remove_hoh and residue.get_resname() == 'HOH':
                            continue
                        protein_seq.append(residue.get_resname())
                        for atom in residue:
                            # 删除氢原子
                            atom_id = atom.get_id()
                            if self.remove_hydrogen and atom.get_id().startswith('H') or pattern.match(atom.get_id()) != None:
                                continue
                            if atom_id.startswith('H') or pattern.match(atom.get_id()) != None:
                                element = 'H'
                            elif atom_id[0:2] in ['CL', 'Cl', 'Br', 'BR', 'AT', 'At']:
                                element = 'Halogen'
                            elif atom_id[0:2] in ['FE', 'ZN', 'MG', 'MN', 'K', 'LI']:
                                element = 'Metal'
                            elif atom_id[0] in ['F', 'I']:
                                element = 'Halogen'
                            elif atom_id[0] in ['C','N','O','S','P']:
                                element = atom_id[0]
                            else:
                                element = atom_id
                            names.append(atom_id)
                            elements.append(element)
                            chain_ids.append(chain.id)
                            resnames.append(residue.get_resname())
                            x, y, z = atom.get_vector()
                            xs.append(x)
                            ys.append(y)
                            zs.append(z)
                protein_df = pd.DataFrame({'chain': chain_ids, 'resname': resnames, 'element': elements, 'name': names, 'x': xs, 'y': ys, 'z': zs})
                processed_complex['atoms_protein'] = protein_df
                processed_complex['protein_seq'] = protein_seq
                # print(processed_complex)
                self.processed_complexes.append(processed_complex)
                count += 1
                # print("count:", count)
                # print(len(processed_complexes))
                # if count == 128:
                #     break
            print(len(self.processed_complexes))
            print("Structure processed Done, dumping...")
            print("Structures with Wrong numbers:", len(wrong_number), wrong_number)
            print("Structures with NMR methods:", len(nmr_files), nmr_files)
            print("Corrupted:", len(corrupted), corrupted)
            pickle.dump(self.processed_complexes, open(self.cache_dir, 'wb'))
            print("Complexes Before Checking:", self.len())
            self.check_dataset()
            print("Complexes Before Task Selection:", self.len())
            self.choose_task_items()
            print("Dataset size:", self.len())
            if self.alpha_only:
                print("Only retaining Alpha Carbon atoms for the atom_df")
                self.retain_alpha_carbon()
    def correctness_check(self, chain_uniprot_info, complx):
        #Annotation Correctness Check
        correct = True
        chain_ids = list(set(complx['atoms_protein']['chain']))
        if '-' not in complx['complex_id']:
            # if complx['complex_id'] == '4p3y':
            #     print(complx['complex_id'], id, uniprot_id, chain_ids, chain_uniprot_info[complx['complex_id']])
            #     print(labels_uniprot)
            for i, id in enumerate(chain_ids):
                if id in chain_uniprot_info[complx['complex_id']]:
                    uniprot_id = chain_uniprot_info[complx['complex_id']][id]
                    labels_uniprot = complx['labels']['uniprots']
                    if uniprot_id not in labels_uniprot:
                        print("Error, you shouldn't come here!")
                        correct = False
                        print(complx['complex_id'],id, uniprot_id, chain_ids, chain_uniprot_info[complx['complex_id']])
                        print(labels_uniprot)
        return correct
    def cal_length_thres(self, complxes):
        length_list = []
        for complx in complxes:
            length = len(complx['atoms_protein']['element'])
            length_list.append(length)
        sorted_list = sorted(length_list)
        thres = sorted_list[int(0.95*len(sorted_list))]
        print("Cutting Threshold of atom numbers:", thres)
        return thres
    def length_check(self, complx, thres):
        if len(complx['atoms_protein']['element']) > thres:
            return False
        else:
            return True
    def check_dataset(self):
        print("Checking the dataset...")
        info_root = './output_info/uniprot_dict_all_go.json'
        with open(info_root, 'r') as f:
            chain_uniprot_info = json.load(f)
        thres = self.cal_length_thres(self.processed_complexes)
        if self.split == 'train':
            thres = 6712

        self.processed_complexes = [i for i in tqdm(self.processed_complexes) if self.length_check(i, thres) and self.correctness_check(chain_uniprot_info, i)]
    
    def retain_alpha_carbon(self):
        new_complexes = []
        for item in self.processed_complexes:
            protein_df = item['atoms_protein']
            # print("Original Nodes:", len(protein_df))
            new_protein_df = protein_df[protein_df.name == 'CA'].reset_index(drop=True)
            item['atoms_protein'] = new_protein_df
            # print("Retaining Alpha Carbons:", len(new_protein_df))
            new_complexes.append(item)
        self.processed_complexes = new_complexes
    
    def choose_task_items(self):
        # 根据不同的任务，训练单独的模型
        if self.split in ['train', 'val', 'test']:
            extra_dir = './datasets/MultiTask/{}.txt'.format(self.split)
            with open(extra_dir, 'r') as f:
                extra_info = f.readlines()
                extra_info = [i.strip() for i in extra_info]
        else:
            extra_info = []
        if self.task == 'ec':
            print("Using Enzyme Commission Dataset for training:")
            root_dir = './output_info/enzyme_commission_uniprots.json'
            with open(root_dir, 'r') as f:
                info_dict = json.load(f)
            new_complexes = []
            for item in self.processed_complexes:
                if item['complex_id'] in info_dict or item['complex_id'] in extra_info: #.keys()?
                    labels = item['labels']
                    annot_number = len(labels['uniprots'])
                    for j in range(annot_number):
                        labels['go'][j] = -1
                    item['labels'] = labels
                    new_complexes.append(item)
            self.processed_complexes = new_complexes
            self.transform_func = GNNTransformEC(task=self.task, gearnet=self.gearnet)
            print("Using EC dataset and transformation")
        elif self.task in ['bp', 'mf', 'cc', 'go']:
            print("Using Gene Ontology {} Dataset for training:".format(self.split))
            # root_dir = './output_info/gene_ontology_uniprots.json'
            root_dir = './output_info/go_task_uniprots.json'
            with open(root_dir, 'r') as f:
                info_dict = json.load(f)
            new_complexes = []
            print("ok")
            for item in self.processed_complexes:
                if item['complex_id'] in info_dict or item['complex_id'] in extra_info:
                    labels = item['labels']
                    annot_number = len(labels['uniprots'])
                    for j in range(annot_number):
                        labels['ec'][j] = -1
                    item['labels'] = labels
                    new_complexes.append(item)
            self.processed_complexes = new_complexes
            print([complx['complex_id'] for complx in new_complexes])
            self.transform_func = GNNTransformGO(task=self.task, gearnet=self.gearnet)
        elif self.task in ['affinity', 'lba', 'ppi']:
            print("Using Affinity Dataset for trainisng:")
            root_dir = './output_info/protein_protein_uniprots.json'
            with open(root_dir, 'r') as f:
                info_dict1 = json.load(f)
            root_dir2 = './output_info/protein_ligand_uniprots.json'
            with open(root_dir2, 'r') as f:
                info_dict2 = json.load(f)
            info_dict = {**info_dict1, **info_dict2}
            new_complexes = []
            for item in self.processed_complexes:
                if item['complex_id'] in info_dict or item['complex_id'] in extra_info:
                    labels = item['labels']
                    annot_number = len(labels['uniprots'])
                    for j in range(annot_number):
                        labels['ec'][j] = -1
                        labels['go'][j] = -1
                    item['labels'] = labels
                    new_complexes.append(item)
            self.processed_complexes = new_complexes
            # print(new_complexes)
            self.transform_func = GNNTransformAffinity(task=self.task, gearnet=self.gearnet)
        else:
            print([complx['complex_id'] for complx in self.processed_complexes])
            self.transform_func = GNNTransformMultiTask(gearnet=self.gearnet)
    def len(self):
        return len(self.processed_complexes)
    def get(self, idx):
        # print(type(self.processed_complexes))
        return self.transform_func(self.processed_complexes[idx])

In [155]:
# from utils.multitask_data import CustomMultiTaskDataset
r_d = './datasets/MultiTask_new/'
_ = CustomMultiTaskDataset(split='val', task='multi', root_dir=r_d, label_dir='./datasets/MultiTask_new/uniformed_labels.json')

in
in Initializing MultiTask Dataset...
Start loading cached Multitask files...
Complexes Before Checking: 283
Checking the dataset...
Cutting Threshold of atom numbers: 4873


100%|██████████| 283/283 [00:00<00:00, 1518.56it/s]


Complexes Before Task Selection: 271
['5e13', '4m2v', '4q81', '3ffp', '1hn4', '3mmf', '3mna', '1f0t', '1o35', '3bgq', '1xq0', '4q7w', '1z6s', '4km0', '1o0h', '1qb6', '5u0g', '2zq2', '3dd8', '3ikd', '3axz', '1tng', '4hp0', '6got', '2zdn', '4tun', '6guk', '3myq', '3aas', '1zge', '3ml2', '1jys', '4k0y', '3rz1', '5u0d', '1o2q', '4aba', '2c3i', '3ibn', '3mhm', '1g36', '5fng', '1c5s', '4qgd', '1g53', '5o07', '1eb2', '5tya', '5uln', '2x7u', '2v2c', '1gi1', '1k1o', '1o0n', '3ibi', '4zx0', '1v2j', '4z1k', '1o2h', '2o4z', '6gxb', '6no9', '4qy3', '4jsa', '3ljo', '1bty', '5dwr', '1bnv', '1g45', '3k8q', '1v2o', '2x7t', '4abf', '6q4e', '3iog', '4u5s', '6dpx', '6h34', '1jsv', '2xog', '1gj6', '5sz7', '6ql2', '5e2r', '4ase', '1okl', '4bf6', '3t82', '6c7x', '6n0k', '5dpx', '1i9n', '1v2t', '3m1k', '6hgr', '5e2k', '1fki', '1bnn', '3n86', '4m2u', '2bza', '1fv0', '4z1e', '5eh8', '1o3j', '5n0e', '3b4f', '3mho', '4u5o', '1mrs', '6b59', '4yyt', '1o3f', '5ipj', '3d6o', '6ugr', '3gy3', '1s5z', '1v2w', '4tmk', '1

In [162]:
_.processed_complexes[17]['atoms_protein']['chain']

0       A
1       A
2       A
3       A
4       A
       ..
2803    A
2804    A
2805    A
2806    A
2807    A
Name: chain, Length: 2808, dtype: object

In [157]:
import json
from tqdm import tqdm
test_list = ['1qbn', '1qhc', '4rfc', '3mhc', '5ty9', '5vgy', '2xj1', '4b6o', '5sz2', '4ag8', '2gkl', '3jya', '4m2w', '5mmg', '4q8x', '5ep7', '4gu6', '1f74', '1v2k', '2gss', '1bnq', '3s75', '1fkg', '3tmk', '3mfv', '4kiu', '3a1e', '1k1m', '1kpm', '3ryj', '3oku', '5th4', '1bn1', '5ngz', '5j41', '4e3g', '4q09', '5mme', '1w4p', '4g8v', '4cp5', '1xug', '3s76', '1u1b', '6h2z', '4agc', '3p58', '1rpf', '1bzy', '2on6', '3d6p', '5flt', '1cny', '4g90', '6ic2', '3n8k', '4qer', '4q7s', '3kv2', '1f73', '3mjl', '3d8z', '3daz', '4q9y', '1qbo', '6hgs', '6ecz', '1c5t', '1fkh', '1fkb', '1drv', '3vhk', '2arm', '3v7x', '1rnm', '1jq8', '4q99', '4kao', '5tb6', '1ydk', '4bqs', '6gxe', '4r5b', '5f2u', '2hoc', '3gy4', '5nya', '4z1j', '3dbu', '5sz0', '5sz5', '5ljt', '5fdi', '3a1d', '6pl1', '2q1q', '1g52', '5lle', '3acl', '3fuc', '1rr6', '1lbk', '2q38', '6pvv', '3ibl', '1xgi', '4unp', '3n0n', '1uto', '1tnh', '2zdl', '1ctt', '1xpz', '2ez7', '3m40', '5amg', '1cnx', '4g95', '1w4o', '3n76', '5dbm', '3hkt', '2hnc', '1o36', '3nkk', '3bl1', '1gar', '4abd', '3dd0', '2hl4', '2xj2', '1afk', '1gi4', '3rz7', '6q4g', '5ny1', '4ruy', '2zfs', '1g4o', '2zdk', '3m67', '5nxw', '3p5l', '1uho', '5n25', '2weg', '1ttm', '1v2l', '1m2x', '6guh', '3n2p', '2euk', '3n3j', '5eh5', '5j8z', '1pxn', '1hi4', '3ibu', '4rsk', '5ny3', '5nxo', '5evd', '5tmp', '5n24', '5llc', '1g48', '1if8', '6oe0', '3m96', '3bgz', '4ciw', '3t5u', '2vxn', '1c5q', '1ppc', '1e1v', '1b8n', '6n0j', '1ydb', '2xb8', '5ekm', '4yk0', '1afl', '2pov', '1g3d', '1w4q', '4rux', '1a1e', '2qdt', '1avn', '1yda', '4gu9', '1kv5', '1if7', '3oim', '5h85', '3kwa', '2fu8', '5ehv', '5mn1', '4abh', '2i6b', '4pmm', '1utn', '3bgs', '6cjv', '4q08', '5u0e', '4hj2', '3ryv', '6ugp', '1o2w', '4kz4', '4yml', '2bes', '1bcd', '4q7p', '1o38', '1b8o', '1c1r', '1y3v', '1amk', '1pxo', '3s78', '1y3x', '1iih', '2weo', '6of5', '6equ', '5ljq', '2v2h', '5e2s', '5fdc', '5nee', '3dcc', '4kni', '5etj', '1o33', '6ugn', '1qb9', '3ryx', '5flo', '3oil', '3d8w', '4n6z', '1dud', '1ql7', '1o0m', '6c7w', '6uh0', '2exm', '5sz3', '1i9p', '4q7v', '1pxp', '5mpz', '1zvy', '3ukw', '1bvk', '1kxt', '2iff', '1zmy', '3a6b', '2p48', '2p46', '3qsk', '4glv', '1zv5', '2p43', '4gn5']
train_list = ['1g52', '4rfc', '3mfv', '1bju', '1sv3', '3t85', '1o2o', '2weo', '1hi3', '4abg', '2x7t', '1jvu', '1okl', '3iog', '4n6z', '3n8k', '6uh0', '1eb2', '4hj2', '1o3d', '5evd', '1ucn', '5jsq', '1b38', '2gkl', '1uho', '5ny1', '3oyq', '2weg', '4n8q', '5u0f', '3s73', '2p48', '3a6b']
val_list = ['5e13', '4m2v', '4q81', '3ffp', '1hn4', '3mmf', '3mna', '1f0t', '1o35', '3bgq', '1xq0', '4q7w', '1z6s', '4km0', '1o0h', '1qb6', '5u0g', '2zq2', '3dd8', '3ikd', '3axz', '1tng', '4hp0', '6got', '2zdn', '4tun', '6guk', '3myq', '3aas', '1zge', '3ml2', '1jys', '4k0y', '3rz1', '5u0d', '1o2q', '4aba', '2c3i', '3ibn', '3mhm', '1g36', '5fng', '1c5s', '4qgd', '1g53', '5o07', '1eb2', '5tya', '5uln', '2x7u', '2v2c', '1gi1', '1k1o', '1o0n', '3ibi', '4zx0', '1v2j', '4z1k', '1o2h', '2o4z', '6gxb', '6no9', '4qy3', '4jsa', '3ljo', '1bty', '5dwr', '1bnv', '1g45', '3k8q', '1v2o', '2x7t', '4abf', '6q4e', '3iog', '4u5s', '6dpx', '6h34', '1jsv', '2xog', '1gj6', '5sz7', '6ql2', '5e2r', '4ase', '1okl', '4bf6', '3t82', '6c7x', '6n0k', '5dpx', '1i9n', '1v2t', '3m1k', '6hgr', '5e2k', '1fki', '1bnn', '3n86', '4m2u', '2bza', '1fv0', '4z1e', '5eh8', '1o3j', '5n0e', '3b4f', '3mho', '4u5o', '1mrs', '6b59', '4yyt', '1o3f', '5ipj', '3d6o', '6ugr', '3gy3', '1s5z', '1v2w', '4tmk', '1o0f', '1k1n', '5wgp', '1ndz', '1y6r', '1bnu', '3f8e', '3oaf', '1mrn', '6oe1', '1o2j', '5evk', '4rfd', '1o2o', '4k18', '3n7a', '2fvd', '1lvu', '4asd', '3lp4', '3s72', '3p4v', '3t83', '1j4r', '1g54', '4ab9', '1o2r', '6h33', '1tx7', '1c5p', '5llg', '1b57', '4yxi', '1k1j', '2j27', '1ctu', '1jn4', '1tni', '4abg', '3h1x', '6h29', '6dpy', '1zfq', '5txy', '3ni5', '5j0d', '5llh', '4r5a', '5evb', '4kz3', '5mpn', '3f80', '10gs', '3m3x', '1g3e', '1bju', '1ndy', '3gst', '4abb', '5amd', '2r9x', '1pph', '4mo8', '1a9q', '3hkn', '4eb8', '2pow', '3r17', '6g3q', '4abe', '4qf7', '5n1r', '1k1l', '1jvu', '5nlk', '1hi5', '2fx6', '2gst', '5ti0', '4jsz', '6dpz', '3hs4', '1o3d', '5wex', '2xp7', '4q83', '2weh', '6ebe', '4qf8', '1t5f', '2w5g', '4iwz', '3d9z', '4yx4', '3qlm', '1hi3', '1bn4', '3s71', '3jy0', '2zdm', '4r59', '3n4b', '3nb5', '4q6d', '4rn4', '1oxr', '1o2n', '1o3i', '1bnt', '5u0f', '4ht0', '3eft', '4hpi', '3oyq', '4k9y', '3vbd', '4z0q', '1f0u', '1b38', '5mnn', '4zx1', '5mpk', '3aau', '4kz6', '6g6t', '5nxp', '3m5e', '5l2s', '3iof', '3mhl', '5x74', '3ryz', '6ibk', '1vfn', '4bf1', '4x8o', '2ot1', '1o2z', '6jb2', '1rjc', '2p45', '1kxv', '4pgj', '2p42', '3ukx', '3a6c', '4pou', '2p44', '2p47', '4gn3', '4gn4']
train_all_list = [cpl['complex_id'] for cpl in _.processed_complexes]
lines = []
with open('./datasets/MultiTask_new/test.txt') as f:
    lines.extend(f.readlines())
with open('./datasets/MultiTask_new/train.txt') as f:
    lines.extend(f.readlines())
with open('./datasets/MultiTask_new/val.txt') as f:
    lines.extend(f.readlines())
with open('./datasets/MultiTask_new/train_all.txt') as f:
    lines.extend(f.readlines())
with open('./datasets/MultiTask_new/uniformed_labels.json') as f:
    labels = json.load(f)
for line in tqdm(lines):
    if len(labels[line[:-1]]['uniprots']) > 1 and line[:-1] in test_list+ val_list + train_list + train_all_list:
        print(line)

100%|██████████| 45873/45873 [00:00<00:00, 215609.65it/s]


In [148]:
for idx, pdb in enumerate(test_list):
    if pdb == '3po1':
        print(idx, pdb)

In [149]:
_.choose_task_items()

['5e13', '4m2v', '4q81', '3ffp', '1hn4', '3mmf', '3mna', '1f0t', '1o35', '3bgq', '1xq0', '4q7w', '1z6s', '4km0', '1o0h', '1qb6', '5u0g', '2zq2', '3dd8', '3ikd', '3axz', '1tng', '4hp0', '6got', '2zdn', '4tun', '6guk', '3myq', '3aas', '1zge', '3ml2', '1jys', '4k0y', '3rz1', '5u0d', '1o2q', '4aba', '2c3i', '3ibn', '3mhm', '1g36', '5fng', '1c5s', '4qgd', '1g53', '5o07', '1eb2', '5tya', '5uln', '2x7u', '2v2c', '1gi1', '1k1o', '1o0n', '3ibi', '4zx0', '1v2j', '4z1k', '1o2h', '2o4z', '6gxb', '6no9', '4qy3', '4jsa', '3ljo', '1bty', '5dwr', '1bnv', '1g45', '3k8q', '1v2o', '2x7t', '4abf', '6q4e', '3iog', '4u5s', '6dpx', '6h34', '1jsv', '2xog', '1gj6', '5sz7', '6ql2', '5e2r', '4ase', '1okl', '4bf6', '3t82', '6c7x', '6n0k', '5dpx', '1i9n', '1v2t', '3m1k', '6hgr', '5e2k', '1fki', '1bnn', '3n86', '4m2u', '2bza', '1fv0', '4z1e', '5eh8', '1o3j', '5n0e', '3b4f', '3mho', '4u5o', '1mrs', '6b59', '4yyt', '1o3f', '5ipj', '3d6o', '6ugr', '3gy3', '1s5z', '1v2w', '4tmk', '1o0f', '1k1n', '5wgp', '1ndz', '1y6r',

In [158]:
complex_ = _.get(17)

Using Transform LBA
atom_df shape:  (2875, 7)
protein_df shape:  (2808, 7)
ligand_df shape:  (67, 4)
atom_df shape:  (1655, 7)
lig_flag shape:  torch.Size([1655])
     chain resname element name          x      y      z
0        A     ILE       N    N  18.568001  5.023  8.831
1        A     ILE       C   CA  18.843000  5.109  7.372
2        A     ILE       C    C  20.315001  5.484  7.189
3        A     ILE       O    O  20.705000  6.540  7.715
4        A     ILE       C   CB  17.920000  6.103  6.679
...    ...     ...     ...  ...        ...    ...    ...
1650   NaN     NaN       C  NaN  19.008000 -3.505  6.891
1651   NaN     NaN       N  NaN  18.194000 -3.133  5.902
1652   NaN     NaN       N  NaN  20.318000 -3.639  6.717
1653   NaN     NaN       C  NaN  17.222000 -3.065  8.612
1654   NaN     NaN       C  NaN  16.620000 -3.292  9.843

[1655 rows x 7 columns]
torch.Size([1625])
torch.Size([1655])
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [82]:
complex_

MyData(x=[242], edge_index=[2, 3450], pos=[242, 3], edge_weights=[3450], affinities=[1, 2], affinity_mask=[1, 2], functions=[1, 3293], valid_masks=[1, 4], chains=[222], lig_flag=[242], prot_id='3po1', type='multi')

In [50]:
print(my_functions)

None


In [51]:
prop = torch.zeros(2755)
prop.scatter_(0,)

TypeError: scatter_() received an invalid combination of arguments - got (int), but expected one of:
 * (int dim, Tensor index, Tensor src)
 * (int dim, Tensor index, Tensor src, *, str reduce)
 * (int dim, Tensor index, Number value)
 * (int dim, Tensor index, Number value, *, str reduce)


In [66]:
sdim= 100,
vdim: int = 16,
depth: int = 3,
r_cutoff: float = 5.0,
num_radial: int = 32,
model_type: str = "eqgat",
learning_rate: float = 1e-4,
weight_decay: float = 0.0,
patience_scheduler: int = 10,
factor_scheduler: float = 0.75,
max_epochs: int = 30,
use_norm: bool = True,
aggr: str = "mean",
enhanced: bool = False,
offset_strategy: int = 0,
task = 'multi'

In [67]:
import torch
import math
from torch import Tensor, nn
from gmsl.model import BaseModel, SEGNNModel
from torch_geometric.data import Batch
from typing import Tuple
model = BaseModel(sdim=100,
                                   vdim=16,
                                   depth=3,
                                   r_cutoff=r_cutoff,
                                   num_radial=32,
                                   model_type="gearnet",
                                   graph_level=True,
                                   num_elements=10,
                                   out_units=1,
                                   dropout=0.0,
                                   use_norm=use_norm,
                                   aggr=aggr,
                                   cross_ablate=False,
                                   no_feat_attn=False,
                                   task=task,
                                   readout='vanilla'
                                #    protein_function_class_dims = class_dims
                                   )

self.task: multi
Readout Strategy: vanilla


In [68]:
train_dataset = CustomMultiTaskDataset(split='train', task='multi', gearnet=True, 
                                       alpha_only=True)

in
in Initializing MultiTask Dataset...
Start loading cached Multitask files...
Complexes Before Checking: 264
Checking the dataset...
Cutting Threshold of atom numbers: 6087


100%|██████████| 264/264 [00:00<00:00, 1377.34it/s]


Complexes Before Task Selection: 254
['1o2o', '1g52', '6pvv', '1ppm', '2zdm', '4jsa', '3lxe', '5i8g', '3jdw', '1o2z', '1hfs', '6ql1', '4k0y', '1pme', '1jsv', '3huc', '2bal', '4myd', '2qrk', '4n8q', '4i71', '1o3l', '3tmk', '1ciz', '5gmn', '4e4n', '3twp', '1uho', '2nn7', '5j7q', '2c97', '4kiu', '5v0n', '1c5q', '5ndf', '4rfc', '6ebe', '2xj2', '5jsq', '3lpi', '4oc0', '3ljo', '1apv', '2uz9', '2x7t', '2hu6', '1o3i', '4e4l', '3m35', '4fai', '5flo', '5aml', '3lk8', '1kyv', '4en4', '1ado', '5ia1', '1dud', '5ny1', '1mrn', '5eng', '3ckz', '6dpz', '3f5j', '3ldp', '6g3q', '6got', '4wop', '3s76', '3bl1', '3mho', '4oc2', '5fck', '6dz0', '1bnu', '6ftz', '3hs4', '4hj2', '2v59', '6p86', '3buf', '4gfm', '6gjm', '2weg', '6h37', '4qem', '3cd5', '4pv5', '4h7q', '3pbb', '3gcs', '2weo', '6nw3', '5sz4', '1jvu', '1ogd', '3d8z', '3ryj', '2a14', '3bgz', '4efs', '3s73', '4k18', '1if8', '5i3v', '4wrb', '5n1r', '1o0n', '3mfv', '1lbk', '1d7j', '2bes', '1bju', '3hub', '6h34', '5sz2', '4qlk', '1eb2', '4e3g', '4gr3', '4

In [69]:
train_loader = DataLoader(
            train_dataset,
            batch_size=32,
            shuffle=True,
            num_workers=4,
            pin_memory=True,
            persistent_workers=True,
        )

In [71]:
y = None
d = None
for idx, data in enumerate(train_loader):
    y = model(data)
    # d = data

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home-new/rhan21/mambaforge/envs/gmsl/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home-new/rhan21/mambaforge/envs/gmsl/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home-new/rhan21/mambaforge/envs/gmsl/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home-new/rhan21/mambaforge/envs/gmsl/lib/python3.9/site-packages/torch_geometric/data/dataset.py", line 258, in __getitem__
    data = self.get(self.indices()[idx])
  File "/tmp/ipykernel_2170304/83550311.py", line 389, in get
    return self.transform_func(self.processed_complexes[idx])
  File "/tmp/ipykernel_2170304/1926787363

In [63]:
y[2].shape
y_ = torch.hstack(y)

In [64]:
y_.shape

torch.Size([8, 2755])

In [81]:
d.y.view(-1,)

tensor([ 3.7000,  7.1700, 10.1700,  6.5500,  8.5200,  8.0100,  7.5100])

In [89]:
d[0]

MyData(x=[279, 21], pos=[279, 3], num_residues=[1], chain=0        A
1        A
2        A
3        A
4        A
      ... 
274    NaN
275    NaN
276    NaN
277    NaN
278    NaN
Name: chain, Length: 279, dtype: object, edge_index=[2, 6613], edge_relations=[6613], edge_weights=[6613], num_relation=[1], functions=[1, 2755], valid_masks=[1, 2755], chains=[257], lig_flag=[279], prot_id='2hl4', num_nodes=279)

In [6]:
import os
import csv
import codecs
root_dir = './data/PDBbind/refined-set'

In [3]:
dir = os.listdir(root_dir)

In [5]:
len(dir)

5318

In [8]:
csv_dir = './data/EC'
rows = []
with codecs.open('./data/EC/pdb_chain_enzyme.csv', encoding='utf-8') as f:
    for row in csv.DictReader(f, skipinitialspace=True):
        rows.append(row)
print(len(rows))

323404


In [9]:
pdbid = []
for row in rows:
    pdbid.append(row['PDB'])

In [10]:
pdbid = list(set(pdbid))

In [11]:
len(pdbid)

110225

In [18]:
ec_dir = os.listdir('./data/EnzymeCommission/all')
len(ec_dir)

19198

In [17]:
pdb_ec = []
for id in ec_dir:
    pdb_ec.append(id[0:4])
pdb_ec = list(set(pdb_ec))
len(pdb_ec)
ec_dic = {}
for id in ec_rows:
    if id[0:4] not in ec_dic.keys():
        ec_dic[id[0:4]] = 

18571

In [28]:
ec_rows = []
with codecs.open('./data/EnzymeCommission/nrPDB-EC_annot_copy.tsv', encoding='utf-8') as f:
    for ec_row in csv.DictReader(f, skipinitialspace=True, delimiter="\t"):
        ec_rows.append(ec_row)
print(len(ec_rows))
for row in ec_rows:
    row['EC-numbers'] =  row['EC-numbers'].split(',')

19201


In [29]:
ec_rows[0]

{'PDB-chain': '4PR3-A', 'EC-numbers': ['3.2.2.9', '3.2.2.-']}

In [31]:
ec_dic = {}
for row in ec_rows:
    if row['PDB-chain'][0:4] not in ec_dic.keys():
        ec_dic[row['PDB-chain'][0:4]] = row['EC-numbers']
    else:
        if ec_dic[row['PDB-chain'][0:4]] != row['EC-numbers']:
            print(row['PDB-chain'][0:4], ec_dic[row['PDB-chain'][0:4]], row['EC-numbers'])

6H8K ['7.1.1.-', '7.1.1.2'] ['1.6.99.-', '1.6.99.3']
3FQD ['3.6.1.-'] ['3.1.13.-']
5GM6 ['3.6.4.13', '3.6.4.-'] ['5.2.1.-', '5.2.1.8']
6F5D ['3.6.3.14', '3.6.3.-'] ['7.1.2.-', '7.1.2.2']
6FGA ['2.3.2.23', '2.3.2.-', '2.3.2.24'] ['2.3.2.27', '2.3.2.-']
5FL7 ['3.6.3.14', '3.6.3.-'] ['7.1.2.-', '7.1.2.2']
5MPC ['3.4.19.12', '3.4.19.-'] ['3.4.25.1', '3.4.25.-']
6RFL ['2.7.7.6', '2.7.7.-'] ['3.6.1.15', '3.6.1.-']
5B8I ['5.2.1.-', '5.2.1.8'] ['3.1.3.16', '3.1.3.-']
6RFQ ['7.1.1.-', '7.1.1.2'] ['1.6.99.3', '1.6.99.-', '7.1.1.-', '7.1.1.2']
2ISS ['3.5.1.-', '4.3.3.6', '3.5.1.2', '4.3.3.-'] ['4.3.3.6', '4.3.3.-']
5XTD ['7.1.1.-', '7.1.1.2'] ['1.6.99.3', '1.6.99.-', '7.1.1.-', '7.1.1.2']
6RFL ['2.7.7.6', '2.7.7.-'] ['2.1.1.-', '2.1.1.56']
6G72 ['7.1.1.-', '7.1.1.2'] ['1.6.99.-', '1.6.99.3']
6RFQ ['7.1.1.-', '7.1.1.2'] ['1.6.99.-', '1.6.99.3']
6H8K ['7.1.1.-', '7.1.1.2'] ['1.6.99.-', '1.6.99.3']
4B3J ['2.3.1.-'] ['1.1.1.35', '1.1.1.-']
4PBW ['2.7.10.-', '2.7.10.1'] ['3.1.3.48', '3.1.3.-']
6H8K ['

In [105]:
import torch
a = torch.tensor([[1],
                  [2],
                  [3]])
a.view(-1,)

tensor([1, 2, 3])

In [108]:
my = []
for i in range(3):
    p = torch.tensor([i])
    my.append(p)
torch.vstack(my)

tensor([[0],
        [1],
        [2]])